In [7]:
import pandas as pd
import EDA
from tqdm import tqdm
from EDA import eda
import random
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97272\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\97272\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97272\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import wordnet
synonyms = set()
word = 'shine'
if wordnet.synsets(word):
    for syn in wordnet.synsets(word):
        for l in syn.lemma_names('spa'):
            synonym = l.replace('_', " ").lower()
            synonym = "".join([char for char in synonym if (char.isalpha() or char == ' ')])
            synonyms.add(synonym)
if word in synonyms:
    synonyms.remove(word)
print(synonyms)

{'brillo', 'lucir', 'radiancia espectral', 'relucir', 'resplandor', 'relumbrar', 'esplendor', 'resplandecer', 'brillar', 'reflejar'}


In [9]:
eda(' contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo ')

['contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo',
 'a anular el título de un estudiante privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico superior en Higiene Bucodental sin acudir a claseEl Consejo',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo ']

In [10]:
df = pd.read_json('data/elmoundo_1108.json')

In [11]:
df.head(10)

,category,content
0,internacional,Rusia cierra su embajada ante la OTAN y echará...
1,internacional,Rusia ha decidido suspender la labor de su emb...
2,internacional,"""En respuesta a las acciones de la OTAN, suspe..."
3,internacional,"""Si los miembros de la OTAN tienen algún asunt..."
4,internacional,Este 'portazo' de los rusos tendrá efectos fue...
5,internacional,Corea del Norte lanza un misil balístico desde...
6,internacional,Corea del Norte disparó al mar un misil balíst...
7,internacional,Así avanza la vacunación contra el Covid-19 en...
8,internacional,"A día de hoy, más de 3.749 millones de habita..."
9,internacional,El mundo está embarcado en un proceso de vacun...


In [17]:
content = df['content']
category = df['category']

In [14]:
category_dic = {}
for idx,cate in enumerate(list(set(df['category']))):
    category_dic[cate] = idx

In [15]:
category_dic

{'economia': 0,
 'cultura': 1,
 'television': 2,
 'tecnologia': 3,
 'internacional': 4,
 'ciencia-y-salud': 5,
 'deportes': 6}

In [18]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in category if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

economia:6954
cultura:5689
television:1251
tecnologia:870
internacional:592
ciencia-y-salud:754
deportes:6264
total: 22374
max_cate:6954
min_cate:592
[6954, 5689, 1251, 870, 592, 754, 6264]


#### find a proper increase num_aug

In [19]:
for ca in category_dic.keys():
    p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
    print(category_dic[ca],dic_count[category_dic[ca]],p)

0 6954 1.838530753112289
1 5689 2.2473444994098886
2 1251 10.21993833504625
3 870 14.695566502463054
4 592 21.596525096525095
5 754 16.956422887457368
6 6264 2.041050903119869


In [24]:
augumented_news = [[],[]]

In [25]:
for cate,cont in tqdm(zip(category,content)):
    if len(cont)>30:
        p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
        new_conts = eda(cont,num_aug=max(6,int(p)))
        for new_cont in new_conts:
            augumented_news[0].append(cate)
            augumented_news[1].append(new_cont)
    else:
        augumented_news[0].append(cate)
        augumented_news[1].append(cont)

22374it [07:39, 48.66it/s]


In [26]:
len(augumented_news[0])

152356

In [27]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in augumented_news[0] if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

economia:47264
cultura:38699
television:8516
tecnologia:5966
internacional:4067
ciencia-y-salud:5101
deportes:42743
total: 152356
max_cate:47264
min_cate:4067
[47264, 38699, 8516, 5966, 4067, 5101, 42743]


In [28]:
shuffle_idx = [i for i in range(0,len(augumented_news[0]))]
random.shuffle(shuffle_idx)
shuffled_data = [[],[],[]]
for idx in shuffle_idx:
    shuffled_data[0].append(augumented_news[0][idx])
    shuffled_data[1].append(augumented_news[1][idx])    

In [29]:
augmented_df = {
    'category':shuffled_data[0],
    'content':shuffled_data[1]
}

In [30]:
augmented_df = pd.DataFrame(data=augmented_df)

In [31]:
augmented_df['category'].head(10)

0            cultura
1    ciencia-y-salud
2    ciencia-y-salud
3           economia
4           economia
5           deportes
6            cultura
7           economia
8            cultura
9            cultura
Name: category, dtype: object

In [33]:
augmented_df.to_json('data/augmented_elmundo_slice.json')